In [1]:
import torch
import numpy as np
import scanpy as sc


# 定义GetLoader类，继承Dataset方法，并重写__getitem__()和__len__()方法
class GetLoader(torch.utils.data.Dataset):
	# 初始化函数，得到数据
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一脸懵逼
    def __len__(self):
        return len(self.data)




In [2]:
data=sc.read_h5ad("new_data.h5ad")
data.X.shape

(773, 4576)

In [3]:
type(np.random.rand(10, 20))

numpy.ndarray

In [4]:
type(data.X)

numpy.ndarray

In [5]:
label=data.obs['label'].to_numpy()
label

array(['interneurons', 'interneurons', 'interneurons', 'interneurons',
       'interneurons', 'pyramidal SS', 'pyramidal SS', 'pyramidal SS',
       'pyramidal SS', 'pyramidal SS', 'pyramidal CA1', 'pyramidal CA1',
       'pyramidal CA1', 'pyramidal CA1', 'pyramidal CA1', 'pyramidal CA1',
       'pyramidal CA1', 'pyramidal CA1', 'pyramidal CA1', 'pyramidal CA1',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'oligodendrocytes', 'oligodendrocytes',
       'oligodendrocytes', 'olig

In [6]:
newlabel=[]
for i in label:
    if i=="astrocytes_ependymal":
        newlabel.append(0)
    elif i=="endothelial-mural":
        newlabel.append(1)
    elif i=="interneurons":
        newlabel.append(2)
    elif i=="microglia":
        newlabel.append(3)
    elif i=="oligodendrocytes":
        newlabel.append(4)
    elif i=="pyramidal CA1":
        newlabel.append(5)
    else:
        newlabel.append(6)
np.array(newlabel).shape

(773,)

In [7]:
data.obs

,cell_ids,label,label2,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
52,1772067093_H05,interneurons,Int7,2424,9792.0,0.0,0.0
203,1772063071_A06,interneurons,Int10,2299,6968.0,0.0,0.0
218,1772067065_B12,interneurons,Int15,2162,6247.0,0.0,0.0
271,1772067083_C02,interneurons,Int2,2296,5587.0,0.0,0.0
279,1772067082_B03,interneurons,Int2,2490,5482.0,0.0,0.0
...,...,...,...,...,...,...,...
3000,1772067059_B04,endothelial-mural,Peric,2220,4584.0,0.0,0.0
3001,1772066097_D04,endothelial-mural,Vsmc,1428,2559.0,0.0,0.0
3002,1772063068_D01,endothelial-mural,Vsmc,1976,4014.0,0.0,0.0
3003,1772066098_A12,endothelial-mural,Vsmc,1493,2896.0,0.0,0.0


In [8]:
import pandas as pd
# 把 Pclass 进行独热编码,prefix 参数设置独热编码后的变量名前缀
label_onehot = pd.get_dummies(data.obs.label,prefix='label')
label_onehot.head()

,label_astrocytes_ependymal,label_endothelial-mural,label_interneurons,label_microglia,label_oligodendrocytes,label_pyramidal CA1,label_pyramidal SS
52,0,0,1,0,0,0,0
203,0,0,1,0,0,0,0
218,0,0,1,0,0,0,0
271,0,0,1,0,0,0,0
279,0,0,1,0,0,0,0


In [9]:
torch_data

NameError: name 'torch_data' is not defined

In [10]:
# 随机生成数据，大小为773 * 4576列
source_data = sc.read_h5ad("new_data.h5ad")
# 随机生成标签，大小为10 * 1列
source_label = np.array(newlabel).reshape(773,1)
# 通过GetLoader将数据进行加载，返回Dataset对象，包含data和labels
torch_data = GetLoader(source_data, source_label)

In [13]:
from torch.utils.data import DataLoader

# 读取数据
datas = DataLoader(torch_data, batch_size=6, shuffle=True, drop_last=False, num_workers=200)

D:\anaconda\lib\site-packages\torch\utils\data\dataloader.py:557: UserWarning: This DataLoader will create 200 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [12]:
for i, data in enumerate(datas):
	# i表示第几个batch， data表示该batch对应的数据，包含data和对应的labels
    print("第 {} 个Batch \n{}".format(i, data))